# Workflow for feature testing and comparison

We are testing multiple ways to describe the calibration information in order to find suitable features to solve this classification problem.
Therefore, we have generated a workflow where feature testing can be done, and the modelling performance with (at least) three different classification algorithm are reported in an excel file (modelling_results.csv). To keep the results comparable, the exact same workflow is used for testing in order to avoid reporting differences due to randomization and different splitting of the training and test set.

## Libraries and read in cleaned data

Data cleaning (done by Yvonne) and following steps were taken:
- removing rows with nan in RT
- removing rows with nan in concentration
- removing calibration graphs with only 1 or 2 calibration points

Data set contains 3860 rows and no nan values


In [9]:
# libraries
import pandas as pd
import numpy as np
from plotnine import *

# data
#file_path = "C:/Users/HelenSepman/OneDrive - Kruvelab/Documents/GitHub/ML_calibration_graph_linearity/0_data/data_ready_addfeatures_231122.csv"
##file_path = ".../ML_calibration_graph_linearity/0_data/data_ready_231029.csv"
#df_calibrations = pd.read_csv(file_path)
#df_calibrations.info()

## load data to google colab
from google.colab import files
uploaded = files.upload()



Saving data_ready_addfeatures_231122.csv to data_ready_addfeatures_231122.csv


In [10]:
file_path = "data_ready_addfeatures_231122.csv"
df_calibrations = pd.read_csv(file_path)
df_calibrations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3860 entries, 0 to 3859
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   lab                  3860 non-null   object 
 1   compound             3860 non-null   object 
 2   sample_type          3860 non-null   object 
 3   RT                   3860 non-null   float64
 4   sample               3860 non-null   object 
 5   peak_area            3860 non-null   float64
 6   note                 3860 non-null   object 
 7   c_real_M             3860 non-null   float64
 8   rf                   3860 non-null   float64
 9   rf_error             3860 non-null   float64
 10  slope                3860 non-null   float64
 11  intercept            3860 non-null   float64
 12  residuals            3860 non-null   float64
 13  abs_residuals        3860 non-null   float64
 14  peak_area_norm1      3860 non-null   float64
 15  c_real_M_norm1       3860 non-null   f

## Feature engineering

Define features used for modelling here

In [ ]:
# new features

In [ ]:

# Plotting, if needed
fig = (
    ggplot(data = df_calibrations,
          mapping = aes(x = 'c_real_M', y = 'peak_area')) +
    geom_point(aes(color = "factor(note)")) +
    scale_color_manual(values=("lightgreen", "red")) +
    theme_bw() +
    #scale_y_log10() +
    #scale_x_log10() +

    facet_wrap("compound",
               ncol=4,
               scales="free") +
    theme(figure_size = (16, 30),
          axis_line = element_line(size = 0.5, colour = "black"),
          panel_grid_major = element_line(size = 0.05, colour = "black"),
          panel_grid_minor = element_line(size = 0.05, colour = "black"),
          axis_text = element_text(colour ='black'),
          aspect_ratio=1
          )
)
fig

In [ ]:
# Here we should maybe add the density plots that Yvonne was also showing to show if there is a potential in classification

## Modelling

Using default values here

- Decision Tree
- KNN
- Random Forest
- xgboost?

In [169]:
## Decide on features for modelling
features = ['peak_area','c_real_M']
#features = ['RT','peak_area','c_real_M']
#features = ['RT','peak_area','c_real_M', 'rf', 'rf_error']
#features = ['RT','peak_area','c_real_M', 'rf', 'rf_error', 'slope', 'intercept', 'residuals', 'abs_residuals']
#features = ['RT','peak_area_norm1','c_real_M_norm1', 'rf_norm1', 'rf_error_norm1', 'slope', 'intercept', 'residuals_norm1', 'abs_residuals_norm1'] # best features
#eatures = ['RT','peak_area_norm2','c_real_M_norm2', 'rf_norm2', 'rf_error_norm2', 'slope', 'intercept', 'residuals_norm2', 'abs_residuals_norm2']

In [170]:
# Split dataset into features and target variable
X = df_calibrations[features]
y = df_calibrations[['note']]

In [171]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
np.random.seed(123)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1) # 80% training and 20% test

print(X_train.shape) #(3134, 3)
print(y_train.shape) #(3134, 1)
print(X_test.shape) #(784, 3)
print(y_test.shape) #(784, 1)

(3088, 2)
(3088, 1)
(772, 2)
(772, 1)


## Decision Tree Classifier

In [172]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

np.random.seed(123) # random seed for consistency
DT_model = DecisionTreeClassifier()

# cross-validate
cv_scores1 = cross_val_score(DT_model, X_train, y_train, cv=5)
print(f'Cross-validation scores of DT: {np.mean(cv_scores1)*100}%')

# train classifiers
DT_model.fit(X_train, y_train)

# predict validation set
X_test['DT_note_pred'] = DT_model.predict(X_test[features])

print(f"Accuracy of DT on test set: {DT_model.score(X_test[features], y_test[['note']])*100}%")  #['RT','peak_area','c_real_M']



Cross-validation scores of DT: 55.01932830849763%
Accuracy of DT on test set: 51.16580310880829%


## KNN

In [173]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

np.random.seed(123) # random seed for consistency
KNN_model = KNeighborsClassifier()

# cross-validate
cv_scores2 = cross_val_score(KNN_model, X_train, y_train['note'], cv=5)
print(f'Cross-validation scores of KNN: {np.mean(cv_scores2)*100}%')

# train classifiers
KNN_model.fit(X_train, y_train['note'])

# predict validation set
X_test['KNN_note_pred'] = KNN_model.predict(X_test[features])

print(f"Accuracy of KNN on test set: {KNN_model.score(X_test[features], y_test[['note']])*100}%")


Cross-validation scores of KNN: 52.42907271325392%
Accuracy of KNN on test set: 51.554404145077726%


## Random forest

In [175]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

np.random.seed(123) # random seed for consistency
RF_model = RandomForestClassifier()

# cross-validate
cv_scores3 = cross_val_score(RF_model, X_train, y_train['note'], cv=5)
print(f'Cross-validation scores of RF: {np.mean(cv_scores3)*100}%')

# train classifiers
RF_model.fit(X_train, y_train['note'])

# predict validation set
X_test['RF_note_pred'] = RF_model.predict(X_test[features])

print(f"Accuracy of RF on test set: {RF_model.score(X_test[features], y_test[['note']])*100}%")

Cross-validation scores of RF: 58.32229233214269%
Accuracy of RF on test set: 60.62176165803109%


## Extreme gradient boosting

In [176]:
#pip install -q xgboost
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Create a XGBoost Classifier
np.random.seed(123)
xgboost = xgb.XGBClassifier(n_estimators=100)

# cross-validate
cv_scores4 = cross_val_score(xgboost, X_train, y_train['note'], cv=5)
print(f'Cross-validation scores of xgboost: {np.mean(cv_scores4)*100}%')

# Train the model using the training sets y_pred=xgboost.predict(X_test)
xgboost.fit(X_train,y_train['note'])

X_test['xgboost_note_pred'] = xgboost.predict(X_test[features])

print(f"Accuracy of xgboost on test set: {xgboost.score(X_test[features], y_test[['note']])*100}%")

Cross-validation scores of xgboost: 69.39754422956891%
Accuracy of xgboost on test set: 67.09844559585493%


## Improving the ensemble models

In [186]:
# I am struggling getting this one working, but Gordian was using some other XGBoost that did not need all the preprocessing?

import xgboost as xgb
from sklearn.metrics import accuracy_score

# transform data
dtrain = xgb.DMatrix(X_train, label=y_train['note'], enable_categorical = True)
dtest = xgb.DMatrix(X_test, label=y_test['note'], enable_categorical = True)

# Define the XGBoost parameters
params = {
    "objective": "binary:logistic",
    #"num_class": 2,   ## This code works, when taking out num_class, not required for binary class problem, but gives an accuracy of 0%.
    "eval_metric": ["error"]
}


np.random.seed(123) # random seed for consistency
xgboost = xgb.train(params, dtrain, num_boost_round = 25)

X_test['xgboost_note_pred'] = xgboost.predict(X_test[features])

threshold = 0.5
preds_binary = [1 if i > threshold else 0 for i in preds]

print(f"Accuracy of xgboost on test set: {xgboost.score(preds_binary, y_test[['note']])*100}%")

TypeError: ignored

In [49]:
! pip install bayesian-optimization

In [96]:
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Define the function to optimize
def bo_tune_xgb(max_depth, gamma, n_estimators ,learning_rate):
    params = {'max_depth': int(max_depth),
              'gamma': gamma,
              'n_estimators': int(n_estimators),
              'learning_rate':learning_rate,
              'subsample': 0.8,
              'eta': 0.1,
              'eval_metric': 'rmse'}
    cv_result = xgb.cv(params, dtrain, num_boost_round=50, nfold=5)
    return -1.0 * cv_result['test-rmse-mean'].iloc[-1]

# Initialize the optimizer
optimizer = BayesianOptimization(bo_tune_xgb, {'max_depth': (3, 10),
                                               'gamma': (0, 1),
                                               'learning_rate':(0,1),
                                               'n_estimators':(10,120)},
                                 random_state=0)

# Optimize
optimizer.maximize(n_iter=50, init_points=8)

# Extract the best parameters
params = optimizer.max['params']
params['max_depth'] = int(params['max_depth'])
# params['learning_rate'] = int(params['learning_rate']) with this 58.8% accuracy
params['n_estimators'] = int(params['n_estimators'])
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'error'

# Train the model with the optimal parameters
XGB_model = xgb.train(params, dtrain, num_boost_round = 25)

# predict validation set
ypred_proba = XGB_model.predict(dtest)
ypred = [1 if prob > 0.5 else 0 for prob in ypred_proba]

#print(f"The accuracy of XGBoost model is {np.sum(y_test == ypred)/len(y_test)*100}%.")
print("Accuracy of XGBoost on the test set:", accuracy_score(y_test, y_pred))

|   iter    |  target   |   gamma   | learni... | max_depth | n_esti... |
-------------------------------------------------------------------------


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:29:37] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.



| 1         | -0.3991   | 0.5488    | 0.7152    | 7.219     | 69.94     |


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:29:39] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.



| 2         | -0.3985   | 0.4237    | 0.6459    | 6.063     | 108.1     |


...

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:30:13] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.



| 57        | -0.4401   | 0.3695    | 0.9418    | 8.746     | 117.4     |


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:30:13] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.



| 58        | -0.4133   | 0.4838    | 0.8055    | 8.013     | 52.49     |
Accuracy of XGBoost on the test set: 0.8341968911917098


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [12:30:13] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.



In [23]:
# I am struggling getting this one working, but Gordian was using some other XGBoost that did not need all the preprocessing?

'''

import xgboost as xgb
from sklearn.metrics import accuracy_score

# tranform labels
mapping = {'linear': 1, 'non-linear': 0}
y_train['note'] = y_train['note'].replace(mapping)
y_test['note'] = y_test['note'].replace(mapping)

# transform data
dtrain = xgb.DMatrix(X_train, label=y_train['note'].values.ravel(), enable_categorical = True)
dtest = xgb.DMatrix(X_test, label=y_test['note'].values.ravel(), enable_categorical = True)

# Define the XGBoost parameters
params = {
    "objective": "binary:logistic",
    "num_class": 2,
    "eval_metric": ["error"]
}

# Train the XGBoost model
np.random.seed(123) # random seed for consistency
XGB_model = xgb.train(params, dtrain, num_boost_round = 25)

# predict validation set
ypred = XGB_model.predict(dtest)
print(f"The accuracy of XGBoost model is {np.sum(y_test == ypred)/len(y_test)*100}%.")
'''



'\n\nimport xgboost as xgb\nfrom sklearn.metrics import accuracy_score\n\n# tranform labels\nmapping = {\'linear\': 1, \'non-linear\': 0}\ny_train[\'note\'] = y_train[\'note\'].replace(mapping)\ny_test[\'note\'] = y_test[\'note\'].replace(mapping)\n\n# transform data\ndtrain = xgb.DMatrix(X_train, label=y_train[\'note\'].values.ravel(), enable_categorical = True)\ndtest = xgb.DMatrix(X_test, label=y_test[\'note\'].values.ravel(), enable_categorical = True)\n\n# Define the XGBoost parameters \nparams = {\n    "objective": "binary:logistic",\n    "num_class": 2,\n    "eval_metric": ["error"]\n}\n\n# Train the XGBoost model\nnp.random.seed(123) # random seed for consistency\nXGB_model = xgb.train(params, dtrain, num_boost_round = 25)\n\n# predict validation set\nypred = XGB_model.predict(dtest)\nprint(f"The accuracy of XGBoost model is {np.sum(y_test == ypred)/len(y_test)*100}%.")\n'

In [ ]:
# The plotting could be useful here, but can only be done for data with two variables
# I did not debug this code yet

'''

def draw_points_ggplot2(point_set):
  fig = (
    ggplot(data = point_set,
          mapping = aes(x = 'x1', y = 'x2')) +
    geom_point(aes(colour = 'class',
                   shape = 'class',
                   fill = 'class'),
               size = 5.0,
               stroke = 2.5) +
    labs(
        title ='',
        x = 'x1',
        y = 'x2',
    ) +
    theme_bw() +
    scale_color_manual(['#EC5D57', '#51A7F9']) +
    scale_fill_manual(['#C82506', '#0365C0']) +
    scale_shape_manual(['o', 's']) +
    theme(figure_size = (5, 5),
          axis_line = element_line(size = 0.5, colour = "black"),
          panel_grid_major = element_line(size = 0.05, colour = "black"),
          panel_grid_minor = element_line(size = 0.05, colour = "black"),
          axis_text = element_text(colour ='black'))
  )
  return(fig)

def generate_grid(start, stop, ppu):
  """
  Function that creates data for the
  decision boundary visualisation.
  """
  num_points = (stop - start)*ppu
  x = np.linspace(start, stop, num_points)
  y = np.linspace(start, stop, num_points)
  xx, yy = np.meshgrid(x, y)
  x1, x2 = xx.flatten(), yy.flatten()
  return(pd.DataFrame({'x1':  x1, 'x2': x2}))

start = -3
stop = 4
ppu = 25 # points per unit

grid_data = generate_grid(start, stop, ppu)
print(grid_data.shape) # it should be (19600, 2)

grid_data['model1'] = DT_model.predict(grid_data[['x1', 'x2']])

draw_points_ggplot2(X_test) + geom_point(data = grid_data, mapping = aes(x = 'x1', y = 'x2', colour = 'factor(model1)'),  size = .5, alpha = 0.2) + annotate("text", label = "DT", x = 2.5, y = 3.5, size = 12, colour = "black")
'''

